# **Predict output of new observations**

In [1]:
import pandas as pd
import pickle
import numpy as np

In [ ]:
test_data = pd.read_csv("data/test_data_subset.csv")

In [ ]:
# load the columns to drop file
columns_to_drop=pd.read_csv("columns_to_drop.csv")
columns_to_Retain = set(test_data.columns.values) - set(columns_to_drop.colnames.values)
test_data = test_data[columns_to_Retain]

column_datatypes = test_data.dtypes
categorical_columns = list(column_datatypes[column_datatypes=="object"].index.values)

In [ ]:
# transform the categorical columns by loading the fit encodings on training data

for cf1 in categorical_columns:
    filename = cf1+".sav"
    le = pickle.load(open(filename, 'rb'))
    
    # if an new classes is observed, set it to the 0 class
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    test_data[cf1]=test_data[cf1].apply(lambda x: le_dict.get(x, -1))

In [ ]:
test_data_id = test_data['id']
test_data = test_data.drop('id',axis=1)

In [ ]:
Column_datatypes= test_data.dtypes
Integer_columns = list(Column_datatypes.where(lambda x: x =="int64").dropna().index.values)
# convert the int64 columns categorical
test_data[Integer_columns] = test_data[Integer_columns].astype('category',copy=False)

In [ ]:
# load the saved model and predict on the test data
tunedmodel_rf = pickle.load(open("tunedmodel_rf", 'rb'))

In [ ]:
Y_test_predict = tunedmodel_rf.predict(test_data)
test_data['predict_loss']=Y_test_predict
test_data['id']=test_data_id
test_data = test_data[['id','predict_loss']]

In [ ]:
test_data